# Project 2 - No-show Appointments analysis

This project aims to analyze a No-show medical appointments dataset from Brazil in 2016. Some questions about the dataset are made and going to be analyzed:


* No-show appointments have a higher frequency in a specific month? 
* SMS received and age (usage of technology) are features related to missing an appointment?
* People with more health issues usually go to appointments? 
* Is scholarship a relevant factor? 
* Gender is related to going or missing an appointment? 

Reference day time functions https://dfrieds.com/data-analysis/create-year-month-column.html

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Listing file names in folder
! ls

No-show-med-appointments.ipynb
noshowappointments-may-2016.csv


In [3]:
# Opening csv file with dataset

df_app = pd.read_csv('noshowappointments-may-2016.csv', sep=',')
df_app.head(5)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [4]:
df_app.shape

(110527, 14)

In [5]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [6]:
# Verifying if columns defined as 'object' are strings

print("Gender is", type(df_app['Gender'][0]))
print("Heighbourhood is", type(df_app['Neighbourhood'][0]))
print("No-show is", type(df_app['No-show'][0]))

Gender is <class 'str'>
Heighbourhood is <class 'str'>
No-show is <class 'str'>


The 3 columns above have string values which is ok. 

ScheduledDay and AppointmentDay columns have a clear "mix" of data: time of appointment and day, so a 'split process' will be done later in this columns. Since this project is just going to analyze the AppointmentDay feature, ScheduledDay column will be excluded.

In [7]:
# Excluding ScheduledDay column
del df_app['ScheduledDay']
df_app.head(5)

,PatientId,AppointmentID,Gender,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [8]:
# Checking unique values of No-show column
df_app['No-show'].unique()

array(['No', 'Yes'], dtype=object)

In [9]:
# Checking duplicated rows in dataset
sum(df_app.duplicated())

0

In [10]:
# Checking NaN values in columns
df_app.isna().sum(axis=0)

PatientId         0
AppointmentID     0
Gender            0
AppointmentDay    0
Age               0
Neighbourhood     0
Scholarship       0
Hipertension      0
Diabetes          0
Alcoholism        0
Handcap           0
SMS_received      0
No-show           0
dtype: int64

 No duplicated or NaN values in the dataframe. Next step is to split AppointmentDay column in date and time.

In [11]:
df_app['AppointmentDay'] = pd.to_datetime(df_app['AppointmentDay'])
df_app['AppointmentDay'].head()

0   2016-04-29 00:00:00+00:00
1   2016-04-29 00:00:00+00:00
2   2016-04-29 00:00:00+00:00
3   2016-04-29 00:00:00+00:00
4   2016-04-29 00:00:00+00:00
Name: AppointmentDay, dtype: datetime64[ns, UTC]

In [12]:
df_app['AppointmentTime'] = df_app['AppointmentDay'].dt.time
df_app.head()

,PatientId,AppointmentID,Gender,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,AppointmentTime
0,2.987250e+13,5642903,F,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,No,00:00:00
1,5.589978e+14,5642503,M,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,No,00:00:00
2,4.262962e+12,5642549,F,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,No,00:00:00
3,8.679512e+11,5642828,F,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,00:00:00
4,8.841186e+12,5642494,F,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,No,00:00:00


In [13]:
# Verifying unique values of AppointmentTime
df_app['AppointmentTime'].unique()

array([datetime.time(0, 0)], dtype=object)

Appointment time just have 1 value 00:00, so this data isn't relevant for the anlysis. It will be excluded from the dataset

In [14]:
del df_app['AppointmentTime']
df_app.head(5)

,PatientId,AppointmentID,Gender,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29 00:00:00+00:00,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29 00:00:00+00:00,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29 00:00:00+00:00,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29 00:00:00+00:00,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [15]:
# Transforming AppointmentDay column in a date
df_app['AppointmentDay'] = df_app['AppointmentDay'].dt.date
df_app.head()

,PatientId,AppointmentID,Gender,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [16]:
type(df_app['AppointmentDay'][0])

datetime.date

In [17]:
df_app['AppointmentDay'].unique()

array([datetime.date(2016, 4, 29), datetime.date(2016, 5, 3),
       datetime.date(2016, 5, 10), datetime.date(2016, 5, 17),
       datetime.date(2016, 5, 24), datetime.date(2016, 5, 31),
       datetime.date(2016, 5, 2), datetime.date(2016, 5, 30),
       datetime.date(2016, 5, 16), datetime.date(2016, 5, 4),
       datetime.date(2016, 5, 19), datetime.date(2016, 5, 12),
       datetime.date(2016, 5, 6), datetime.date(2016, 5, 20),
       datetime.date(2016, 5, 5), datetime.date(2016, 5, 13),
       datetime.date(2016, 5, 9), datetime.date(2016, 5, 25),
       datetime.date(2016, 5, 11), datetime.date(2016, 5, 18),
       datetime.date(2016, 5, 14), datetime.date(2016, 6, 2),
       datetime.date(2016, 6, 3), datetime.date(2016, 6, 6),
       datetime.date(2016, 6, 7), datetime.date(2016, 6, 1),
       datetime.date(2016, 6, 8)], dtype=object)

Analyzing the unique values, it is possible to verify that the appointments were scheduled in just 3 months: April, May and June. As the day doesn't matter in this analysis, the AppointmentMonth column will be created and replace the 'AppointmentDay column.

In [18]:
df_app.rename(columns={'AppointmentDay' : 'AppointmentMonth'}, inplace=True)

df_app['AppointmentMonth'] = pd.to_datetime(df_app['AppointmentMonth'])
df_app['AppointmentMonth'] = df_app['AppointmentMonth'].dt.strftime('%m')
df_app.head(5)

,PatientId,AppointmentID,Gender,AppointmentMonth,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,04,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,04,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,04,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,04,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,04,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [19]:
# Checking unique values od AppointmentMonth column
df_app['AppointmentMonth'].unique()

array(['04', '05', '06'], dtype=object)

The PatientId and AppointmentID columns aren't relevant to this analysis, so they will also be excluded

In [20]:
del df_app['PatientId']
del df_app['AppointmentID']

In [21]:
df_app.head(3)

,Gender,AppointmentMonth,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,F,04,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,M,04,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,F,04,62,MATA DA PRAIA,0,0,0,0,0,0,No


## Plotting and Analyzing results

In [ ]:
df_app['Neighbourhood'].unique()